In [1]:
#Credit to Edward Ross - https://skeptric.com/notebooks/Parsing%20Experience%20from%20Adzuna%20Job%20Ads.html
import re
import pandas as pd
import spacy
from spacy.util import filter_spans
from spacy.tokens import Span
from spacy.matcher import Matcher

In [2]:
from spacy import displacy
from IPython.display import HTML, display


In [3]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [4]:
df = pd.read_csv('outputs/data_science_jobs.csv')

In [5]:
df = df.drop(df.columns[0], axis = 1) 

In [6]:
import regex as re
df['Job Description'] = df['Job Description'].apply(lambda x: re.sub(r'[!;,\s]', ' ', x))
df.sample(50, random_state = 5)

,Job Title,Company Name,Job Description,Location,Job Board
852,HEAD OF ANALYTICS & INSIGHTS/DATA SCIENTIST,Saje Natural Wellness,For over 25 years we’ve been helping our commu...,Vancouver,Indeed
207,Machine Learning Engineer,Aviva,About the job Individually we are people but ...,Markham,Indeed
627,"Data Engineer / Power BI Developer, Omnia AI",Deloitte\n,Job Type: Permanent Primary Location: Vancouve...,Vancouver,Glassdoor
671,Junior Business Intelligence Developer - Data ...,CAPRiser Group,Our subsidiary is in PropTech and real estate ...,Toronto,Indeed
661,"Data Scientist Technical Lead (Montreal), Inta...",Intact\n,Who needs insurance? Everybody. That keeps us ...,Montreal,Glassdoor
489,Bio informaticien/Bioinformatics scientist,GenAIz\n,Avant le SRAS-CoV-2 un vaccin qui était dével...,Montreal,Glassdoor
434,Data Scientist,Awakedata,RESPONSIBILITIES: Develop state-of-the-art com...,Burnaby,Glassdoor
448,Scientifique de données / Data Scientist,BusPatrol\n,Qui sommes-nous : BusPatrouille est une entrep...,Montreal,Glassdoor
634,Data Management Specialist - Immunotoxicology,Charles River Laboratories\n,For 70 years Charles River employees have wor...,Senneville,Glassdoor
748,Data Scientist,F8th,We're transforming the Cyber Security industry...,Midtown Toronto,Glassdoor


In [7]:
ads = list(df['Job Description'])

In [8]:
len(ads)

881

In [9]:
def highlight_terms(terms, texts):
    for doc in nlp.pipe(texts):
        for sentence in set([tok.sent for tok in doc if tok.lower_ in terms]):
            text = sentence.text.strip()
            markup = re.sub(fr'(?i)\b({"|".join(terms)})\b', r'<strong>\1</strong>', text)
            display(HTML(markup))
            print('-----')

In [10]:
highlight_terms(['experience'], ads[:10])

-----


-----


-----


-----


-----


...

-----


-----


-----


-----


-----


In [11]:
matcher = Matcher(nlp.vocab)
pattern = [{'POS': 'NOUN', 'OP': '+'}, {'LOWER': 'experience'}]
matcher.add('experience_noun', [pattern])

pattern = [{'LOWER': 'experience'}, {'POS': 'ADP'}, {'POS': {'IN': ('DET', 'NOUN', 'PROPN')}, 'OP': '+'}]
matcher.add('experience_adp', [pattern])

In [12]:
doc = nlp(ads[0])
matcher(doc)

[(12285600890577657150, 117, 119),
 (1417798585642285709, 239, 242),
 (1417798585642285709, 239, 243),
 (12285600890577657150, 265, 267)]

In [13]:
def show_extraction(examples, *extractors):
    seen = set()
    for doc in nlp.pipe(examples):
        doc.ents = filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)])
        for tok in doc:
            if tok.lower_ == 'experience':
                sentence = tok.sent
                if sentence.text in seen:
                    continue
                seen.update([sentence.text])
                if not sentence.ents:
                    doc.ents = list(doc.ents) + [Span(doc, tok.i, tok.i+1, 'MISSING')]
                displacy.render(sentence, style='ent', options = {'colors': {'MISSING': 'pink',
                                                                            'EXPERIENCE': 'lightgreen'}})
                

In [14]:
show_extraction(ads[:5], matcher)


In [15]:
def get_extractions(examples, *extractors):
    # Could use context instead of enumerate
    for idx, doc in enumerate(nlp.pipe(examples, batch_size=100, disable=['ner'])):
        for ent in filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)]):
            sent = ent.root.sent
            yield ent.text, idx, ent.start, ent.end, ent.label_, sent.start, sent.end

In [16]:
list(get_extractions(ads[:3], matcher))


[('user experience', 0, 117, 119, 'experience_noun', 92, 121),
 ('experience in the application', 0, 239, 243, 'experience_adp', 224, 251),
 ('techniques Experience', 0, 265, 267, 'experience_noun', 251, 304),
 ('data science experience', 1, 143, 146, 'experience_noun', 136, 169),
 ('software development experience Experience',
  1,
  157,
  161,
  'experience_noun',
  136,
  169),
 ('Experience with time', 1, 169, 172, 'experience_adp', 169, 178),
 ('environment Experience', 1, 197, 199, 'experience_noun', 178, 212),
 ('fitness experience', 2, 73, 75, 'experience_noun', 70, 76),
 ('experience in ETL', 2, 399, 402, 'experience_adp', 396, 413),
 ('Experience with Informatica IICS', 2, 445, 449, 'experience_adp', 413, 481),
 ('Experience with message queues', 2, 455, 459, 'experience_adp', 413, 481),
 ('experience in a DevOps environment Experience',
  2,
  468,
  474,
  'experience_adp',
  413,
  481)]

In [17]:
def extract_df(*extractors, n_max=None, **kwargs):
    if n_max is None:
        n_max = len(df)
    ent_df = pd.DataFrame(list(get_extractions(df[:n_max]['Job Description'], *extractors)),
                          columns=['text', 'docidx', 'start', 'end', 'label', 'sent_start', 'sent_end'])
    return ent_df.merge(df, how='left', left_on='docidx', right_index=True)

In [18]:
%time 
ent_df = extract_df(matcher, n_max=1000)
ent_df.head()

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.29 µs


,text,docidx,start,end,label,sent_start,sent_end,Job Title,Company Name,Job Description,Location,Job Board
0,user experience,0,117,119,experience_noun,92,121,Data Scientist,Metricsflow Inc,At Metricsflow we work to shape the future of...,St. John's,Glassdoor
1,experience in the application,0,239,243,experience_adp,224,251,Data Scientist,Metricsflow Inc,At Metricsflow we work to shape the future of...,St. John's,Glassdoor
2,techniques Experience,0,265,267,experience_noun,251,304,Data Scientist,Metricsflow Inc,At Metricsflow we work to shape the future of...,St. John's,Glassdoor
3,data science experience,1,143,146,experience_noun,136,169,Data Scientist - 05/12/21,Acerta Analytics Solutions Inc,Acerta’s machine learning platforms leverage a...,Kitchener,Indeed
4,software development experience Experience,1,157,161,experience_noun,136,169,Data Scientist - 05/12/21,Acerta Analytics Solutions Inc,Acerta’s machine learning platforms leverage a...,Kitchener,Indeed


In [19]:
def aggregate_df(df, col=['text']):
    return (df
            .groupby(col)
            .agg(n_company=('Company Name', 'nunique'))
            .reset_index()
            .sort_values(['n_company'], ascending=False)
        )

In [20]:
aggregate_df(ent_df).head(10)


,text,n_company
1369,work experience,49
1383,years experience,45
1203,industry experience,34
619,customer experience,29
793,experience in a,27
1359,user experience,22
870,experience in data science,20
329,Experience with Python,19
1066,experience with Python,15
931,experience in the,15


In [21]:
def showent(docidx, start, end, label, sent_start, sent_end, **kwargs):
    # We don't need to parse it, so just make_doc
    doc = nlp.make_doc(ads[docidx])
    doc.ents = [Span(doc, start, end, label)]
    sent = doc[sent_start:sent_end]
    displacy.render(sent, style='ent')
    
def showent_df(df):
    for idx, row in df.iterrows():
        showent(**row)

In [22]:
showent_df(ent_df.query('text == "experience in a"').head())


In [23]:
def extract_noun_phrase_experience(doc):
    for np in doc.noun_chunks:
        if np[-1].lower_ == 'experience':
            if len(np) > 1:
                yield 'EXPERIENCE', np[0].i, np[-1].i

In [24]:
show_extraction(ads[:5], extract_noun_phrase_experience)


In [25]:
%time
ent_df = extract_df(extract_noun_phrase_experience, n_max=50000)


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


In [26]:
aggregate_df(ent_df).head(50)


,text,n_company
897,hands-on,29
925,industry,28
853,equivalent,25
287,Hands-on,23
1083,relevant,22
253,Extensive,18
1273,years,17
425,Proven,17
875,extensive,16
1265,work,16


In [27]:
experience_qualifiers = ['previous', 'prior', 'following', 'recent', 'the above', 'past',
                         
                         'proven', 'demonstrable', 'demonstrated', 'relevant', 'significant', 'practical',
                         'essential', 'equivalent', 'desirable', 'required', 'considerable', 'similar',
                         'working', 'specific', 'qualified', 'direct', 'hands on', 'handson', 'hands-on'
                         
                         'strong', 'solid', 'good', 'substantial', 'excellent', 'the right', 'valuable', 'invaluable',
                         
                         'some', 'any', 'none', 'much', 'extensive', 'no', 'more',
                         'your', 'their', 'great',
                         'years', 'months',
                        ]

stopwords = ['a', 'an', '*', '**', '.', 'this', 'the', ':', 'Skills']

experience_qualifier_pattern = rf'\b(?:{"|".join(experience_qualifiers)})\b'

experience_qualifier_pattern

'\\b(?:previous|prior|following|recent|the above|past|proven|demonstrable|demonstrated|relevant|significant|practical|essential|equivalent|desirable|required|considerable|similar|working|specific|qualified|direct|hands on|handson|hands-onstrong|solid|good|substantial|excellent|the right|valuable|invaluable|some|any|none|much|extensive|no|more|your|their|great|years|months)\\b'

In [28]:
aggregate_df(ent_df[(~ent_df.text.str.lower().str.contains(experience_qualifier_pattern)) & # Not a qualifier
                     ~ent_df.text.isin(stopwords)]).head(50)

,text,n_company
665,hands-on,29
690,industry,28
191,Hands-on,23
946,work,16
791,professional,15
365,Strong,15
573,customer,13
13,),12
289,Programming,11
953,·,11


In [29]:
def extract_adp_experience(doc, label='EXPERIENCE'):
    for tok in doc:
        if tok.lower_ == 'experience':
            for child in tok.rights:
                if child.dep_ == 'prep':
                    for obj in child.children:
                        if obj.dep_ == 'pobj':
                            yield label, obj.left_edge.i, obj.i+1

In [30]:
show_extraction(ads[10:15], extract_adp_experience)

In [31]:
def extract_adp_experience_2(doc):
    for np in doc.noun_chunks:
        start_tok = np[0].i
        if start_tok >= 2 and doc[start_tok - 2].lower_ == 'experience' and doc[start_tok - 1].pos_ == 'ADP':
            yield 'EXPERIENCE', start_tok, start_tok + len(np)


In [32]:
show_extraction(ads[10:15], extract_adp_experience_2)

In [33]:
%time 
ent_adp_df = extract_df(extract_adp_experience, n_max=50)


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [34]:
%time 
ent_adp_df = extract_df(extract_adp_experience_2, n_max=50)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


In [35]:
%time 
ent_adp_df = extract_df(extract_adp_experience, n_max=50000)


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs


In [36]:
aggregate_df(ent_adp_df).head(50)


,text,n_company
302,Python,48
706,data science,25
673,data,16
328,SQL,14
244,Machine Learning,13
561,big data tools,11
718,data visualization,10
405,a Data Scientist,9
10,AWS,9
676,data analysis,8


In [37]:
showent_df(ent_adp_df.query("text=='C'").head(5))


In [38]:
showent_df(ent_adp_df.query("text=='R'").head(5))


In [39]:
def highlight_text_context(terms, texts, n_before=1, n_after=2):
    context = []
    for doc in nlp.pipe(texts):
        sentences = list(doc.sents)
        idxs = [i for i, sent in enumerate(sentences) if any(term in sent.text.lower() for term in terms)]
        
        for idx in idxs:
            before = ''.join(sent.text for sent in sentences[max(idx-n_before, 0):idx])
            after = ''.join(sent.text for sent in sentences[idx+1:min(idx+n_before+1, len(sentences))])
            text = sentences[idx].text
            markup = re.sub(fr'(?i)\b({"|".join(terms)})\b', r'<strong>\1</strong>',
                                 f'<span style="color:blue">{text}</span>')
            display(HTML(before + markup + after))

In [40]:
terms = ['experience']

for _, q in ent_adp_df.query("text=='a'").head(7).iterrows():
    doc = nlp(q.FullDescription)
    if q.sent_start > 0:
        prev_sent = doc[q.sent_start - 1].sent.text
    else:
        prev_sent = ''
    
    if q.sent_end < len(doc):
        next_sent = doc[q.sent_end].sent.text
    else:
        next_sent = ''
        
    text = doc[q.sent_start:q.sent_end].text
    markup = re.sub(fr'(?i)\b({"|".join(terms)})\b', r'<strong>\1</strong>',
                     f'<span style="color:blue">{text}</span>')
    display(HTML(prev_sent + markup + next_sent))

In [41]:
def get_left_span(tok, label='', include=True):
    offset = 1 if include else 0
    idx = tok.i
    while idx > tok.left_edge.i:
        if tok.doc[idx - 1].pos_ in ('NOUN', 'PROPN', 'ADJ', 'X'):
            idx -= 1
        else:
            break
    return label, idx, tok.i+offset


In [42]:
def get_conjugations(tok):
    new = [tok]
    while new:
        tok = new.pop()
        yield tok
        for child in tok.children:
            if child.dep_ == 'conj':
                new.append(child)

In [43]:
#old
EXP_TERMS = ['experience']
def extract_adp_conj_experience(doc, label='EXPERIENCE'):
    for tok in doc:
        if tok.lower_ in EXP_TERMS:
            for child in tok.rights:
                if child.dep_ == 'prep':
                    for obj in child.children:
                        if obj.dep_ == 'pobj':
                            for conj in get_conjugations(obj):
                                yield get_left_span(conj, label)

In [44]:
show_extraction(ads[10:15], extract_adp_conj_experience)


In [45]:
show_extraction(ads[:10], extract_adp_conj_experience)


In [46]:
def extract_verb_maybeadj_noun_experience(doc, label='EXPERIENCE'):
    for tok in doc:
        if tok.lower_ in EXP_TERMS:
            for child in tok.rights:
                if child.dep_ == 'acl':
                    for gc in child.children:
                        if gc.dep_ == 'prep':
                            for ggc in gc.children:
                                if ggc.dep_ == 'pobj':
                                    for c in get_conjugations(ggc):
                                        yield get_left_span(c, 'EXPERIENCE')
                        elif gc.dep_ == 'dobj':
                            for c in get_conjugations(gc):
                                yield get_left_span(c, 'EXPERIENCE')


In [47]:
show_extraction(ads[5:10], extract_verb_maybeadj_noun_experience)


In [48]:
extract_exps = [extract_adp_conj_experience,]


In [49]:
n_ads = len(df)

In [50]:
%%time
df_ents = extract_df(*extract_exps, n_max=n_ads)

CPU times: user 36.3 s, sys: 1.81 s, total: 38.1 s
Wall time: 38.2 s


In [51]:
df_ents

,text,docidx,start,end,label,sent_start,sent_end,Job Title,Company Name,Job Description,Location,Job Board
0,application,0,242,243,EXPERIENCE,224,251,Data Scientist,Metricsflow Inc,At Metricsflow we work to shape the future of...,St. John's,Glassdoor
1,keras,1,162,163,EXPERIENCE,136,169,Data Scientist - 05/12/21,Acerta Analytics Solutions Inc,Acerta’s machine learning platforms leverage a...,Kitchener,Indeed
2,operations data,1,175,177,EXPERIENCE,169,178,Data Scientist - 05/12/21,Acerta Analytics Solutions Inc,Acerta’s machine learning platforms leverage a...,Kitchener,Indeed
3,data,2,403,404,EXPERIENCE,396,413,Data Engineer,GoodLife Fitness\n,Position Description DATA ENGINEER **This ...,Midtown Toronto,Glassdoor
4,Cloud platforms,2,435,437,EXPERIENCE,413,481,Data Engineer,GoodLife Fitness\n,Position Description DATA ENGINEER **This ...,Midtown Toronto,Glassdoor
...,...,...,...,...,...,...,...,...,...,...,...,...
2464,Data Engineer,879,498,500,EXPERIENCE,485,563,Data Engineer,Cyient\n,Cyient is a global engineering and technology ...,Montreal,Glassdoor
2465,cross-functional agile team,879,502,507,EXPERIENCE,485,563,Data Engineer,Cyient\n,Cyient is a global engineering and technology ...,Montreal,Glassdoor
2466,NoSQL database technologies,880,523,526,EXPERIENCE,515,532,Principal Data Engineer,ELEMENTS Global Services,Elements Global Services is an award-winning H...,Toronto,Indeed
2467,modern programming languages,880,538,541,EXPERIENCE,532,549,Principal Data Engineer,ELEMENTS Global Services,Elements Global Services is an award-winning H...,Toronto,Indeed


In [52]:
showent_df(df_ents[:2])


In [53]:
df_ent_agg = aggregate_df(df_ents)
df_ent_agg.head(10)

,text,n_company
375,Python,56
696,data science,27
652,data,24
407,SQL,23
301,Machine Learning,16
1273,systems,14
212,Experience,14
752,development,13
1298,tools,13
937,machine,12


In [54]:
from flashtext import KeywordProcessor


In [55]:
keyword_processor = KeywordProcessor(case_sensitive=True)


In [56]:
#selecting most popular
skills = df_ent_agg.query('n_company >= 3').text
len(skills)

144

In [57]:
for skill in skills:
    keyword_processor.add_keyword(skill)

In [58]:
from collections import Counter


In [59]:
skills = list(
(df_ent_agg
 .query('n_company >= 3')
).text
)
len(skills)

144

In [60]:
n_max=1000
for a,b,c in zip(skills[:n_max:3],skills[1:n_max:3],skills[2:n_max:3]):
     print('{:<35}{:<35}{:<}'.format(a,b,c))

Python                             data science                       data
SQL                                Machine Learning                   systems
Experience                         development                        tools
machine                            data analysis                      Data Scientist
big data tools                     data visualization                 big data technologies
AWS                                analysis                           NoSQL databases
programming languages              NoSQL                              implementation
customers                          machine learning                   data analytics
Docker                             Spark                              relational databases
field                              AI                                 software development
one                                data pipeline                      design
object function scripting languagesdata visualization tools           technolog

In [61]:
def get_extractions_2(examples, *extractors):
    # Could use context instead of enumerate
    doc = nlp(examples, disable=['ner'])
    for ent in filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)]):
        sent = ent.root.sent
        yield ent.text



In [62]:
def list_skills(examples, *extractors):
    return list(get_extractions_2(examples, *extractors))

In [63]:
print(list_skills(df['Job Description'][10], extract_adp_conj_experience))

['MySQL', 'BigQuery Experience']


In [64]:
#for debugging visually
print(show_extraction(df['Job Description'][10]), extract_adp_conj_experience)

None <function extract_adp_conj_experience at 0x7fe828866700>


In [65]:
%%time
#create a new column in df with the fxn that works
df['skills'] = df['Job Description'].apply(lambda x: list_skills(x, extract_adp_conj_experience)) 


CPU times: user 48.9 s, sys: 236 ms, total: 49.1 s
Wall time: 49.3 s


In [66]:
df.head()

,Job Title,Company Name,Job Description,Location,Job Board,skills
0,Data Scientist,Metricsflow Inc,At Metricsflow we work to shape the future of...,St. John's,Glassdoor,[application]
1,Data Scientist - 05/12/21,Acerta Analytics Solutions Inc,Acerta’s machine learning platforms leverage a...,Kitchener,Indeed,"[keras, operations data]"
2,Data Engineer,GoodLife Fitness\n,Position Description DATA ENGINEER **This ...,Midtown Toronto,Glassdoor,"[data, Cloud platforms, Azure Cloud Data Solut..."
3,Safety Aggregate Reporting & Analytics Coordin...,IQVIA,Job Overview Manage the administrative support...,Kirkland,Indeed,[]
4,Senior Data Engineer,Samsung Electronics\n,Position Summary The SmartThings Big Data tea...,Vancouver,Glassdoor,"[big data engineering, large scale, high avail..."


In [67]:
df.sample(50, random_state = 5)

,Job Title,Company Name,Job Description,Location,Job Board,skills
852,HEAD OF ANALYTICS & INSIGHTS/DATA SCIENTIST,Saje Natural Wellness,For over 25 years we’ve been helping our commu...,Vancouver,Indeed,"[multivariate statistics, media measurement, c..."
207,Machine Learning Engineer,Aviva,About the job Individually we are people but ...,Markham,Indeed,"[API deployment, microservice architecture, sk..."
627,"Data Engineer / Power BI Developer, Omnia AI",Deloitte\n,Job Type: Permanent Primary Location: Vancouve...,Vancouver,Glassdoor,[high proficiency]
671,Junior Business Intelligence Developer - Data ...,CAPRiser Group,Our subsidiary is in PropTech and real estate ...,Toronto,Indeed,"[CSS, JavaScript, JavaScript, one, more, non-r..."
661,"Data Scientist Technical Lead (Montreal), Inta...",Intact\n,Who needs insurance? Everybody. That keeps us ...,Montreal,Glassdoor,"[field, source data mining frameworks]"
489,Bio informaticien/Bioinformatics scientist,GenAIz\n,Avant le SRAS-CoV-2 un vaccin qui était dével...,Montreal,Glassdoor,[multidisciplinary bioinformatics projects Exp...
434,Data Scientist,Awakedata,RESPONSIBILITIES: Develop state-of-the-art com...,Burnaby,Glassdoor,"[big data techniques, relational, NoSQl Databa..."
448,Scientifique de données / Data Scientist,BusPatrol\n,Qui sommes-nous : BusPatrouille est une entrep...,Montreal,Glassdoor,[]
634,Data Management Specialist - Immunotoxicology,Charles River Laboratories\n,For 70 years Charles River employees have wor...,Senneville,Glassdoor,[]
748,Data Scientist,F8th,We're transforming the Cyber Security industry...,Midtown Toronto,Glassdoor,"[statistical package, software development, ma..."


In [68]:
with open('outputs/raw_skills.txt', 'w') as f:
    for skill in skills:
        print(skill, file=f)

In [69]:
df.to_csv('outputs/rule_based_extracted_skills.csv', index=False)
